# Improving performance

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# Load the data
df = pd.read_csv('../data/new_titanic_features.csv')

In [ ]:
# Create Features and Labels
X = df[['Male', 'Family',
        'Pclass2_one', 'Pclass2_two', 'Pclass2_three',
        'Embarked_C', 'Embarked_Q', 'Embarked_S',
        'Age2', 'Fare3_Fare11to50', 'Fare3_Fare51+', 'Fare3_Fare<=10']]
y = df['Survived']


In [ ]:
X.describe()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=.2, random_state=0)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression(solver='liblinear')
model.fit(X_train, y_train)
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
print('Train Accuracy: {:0.3}'.format(accuracy_score(y_train, pred_train)))
print('Test Accuracy: {:0.3}'.format(accuracy_score(y_test, pred_test)))

In [ ]:
confusion_matrix(y_test, pred_test)

In [ ]:
print(classification_report(y_test, pred_test))

## Feature importances (wrong! see exercise 1)

In [ ]:
coeffs = pd.Series(model.coef_.ravel(), index=X.columns)
coeffs

In [ ]:
coeffs.plot(kind='barh')

## Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score, ShuffleSplit

In [ ]:
cv = ShuffleSplit(n_splits=5, test_size=.4, random_state=0)
scores = cross_val_score(model, X, y, cv=cv)
scores

In [ ]:
'Crossval score: %0.3f +/- %0.3f ' % (scores.mean(), scores.std())

## Learning curve

In [ ]:
from sklearn.model_selection import learning_curve

In [ ]:
tsz = np.linspace(0.1, 1, 10)
train_sizes, train_scores, test_scores = learning_curve(model, X, y, train_sizes=tsz, cv=3)

In [ ]:
fig = plt.figure()
plt.plot(train_sizes, train_scores.mean(axis=1), 'ro-', label="Train Scores")
plt.plot(train_sizes, test_scores.mean(axis=1), 'go-', label="Test Scores")
plt.title('Learning Curve: Logistic Regression')
plt.ylim((0.5, 1.0))
plt.legend()
plt.draw()
plt.show()

### Exercise 1

Try rescaling the Age feature with [`preprocessing.StandardScaler`](http://scikit-learn.org/stable/modules/preprocessing.html) so that it will have comparable size to the other features.

- Do the model prediction change?
- Does the performance of the model change?
- Do the feature importances change?
- How can you explain what you've observed?

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [ ]:
sc.fit(X_train[['Age2']])

In [ ]:
X_train_sc = X_train.copy()
X_test_sc = X_test.copy()

X_train_sc['Age2'] = sc.transform(X_train[['Age2']])
X_test_sc['Age2'] = sc.transform(X_test[['Age2']])

In [ ]:
model = LogisticRegression(solver='liblinear')

In [ ]:
model.fit(X_train, y_train)
print('Train Accuracy (not scaled): {:0.3}'.format(accuracy_score(y_train, model.predict(X_train))))
print('Test Accuracy (not scaled): {:0.3}'.format(accuracy_score(y_test, model.predict(X_test))))

coeffs = pd.Series(model.coef_.ravel(), index=X.columns)

In [ ]:
model.fit(X_train_sc, y_train)
print('Train Accuracy (scaled): {:0.3}'.format(accuracy_score(y_train, model.predict(X_train_sc))))
print('Test Accuracy (scaled): {:0.3}'.format(accuracy_score(y_test, model.predict(X_test_sc))))

coeffs_sc = pd.Series(model.coef_.ravel(), index=X.columns)

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(121)
coeffs.plot(kind='barh', title='Unscaled Age2')

plt.subplot(122)
coeffs_sc.plot(kind='barh', title='Scaled Age2')

plt.tight_layout()

Only the coefficients of the rescaled features can be interpreted as feature importances.

### Exercise 2

Experiment with another classifier for example `DecisionTreeClassifier`, `RandomForestClassifier`,  `SVC`, `MLPClassifier`, `SGDClassifier` or any other classifier of choice you can find here: http://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html. 

- Train the model on both the scaled data and on the unscaled data
- Compare the score for the scaled and unscaled data
- how can you get the features importances for tree based models? Check [here](http://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html) for some help.
- Which classifiers are impacted by the age rescale? Why?

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=30)

model.fit(X_train, y_train)
print('Train Accuracy (not scaled): {:0.3}'.format(accuracy_score(y_train, model.predict(X_train))))
print('Test Accuracy (not scaled): {:0.3}'.format(accuracy_score(y_test, model.predict(X_test))))

coeffs = pd.Series(model.feature_importances_, index=X.columns)
coeffs.plot(kind='barh')

In [ ]:
model.fit(X_train_sc, y_train)
print('Train Accuracy (scaled): {:0.3}'.format(accuracy_score(y_train, model.predict(X_train_sc))))
print('Test Accuracy (scaled): {:0.3}'.format(accuracy_score(y_test, model.predict(X_test_sc))))

coeffs = pd.Series(model.feature_importances_, index=X.columns)
coeffs.plot(kind='barh')

### Exercise 3

Pick your preferred classifier from Exercise 2 and search for the best hyperparameters. You can read about hyperparameter search [here](http://scikit-learn.org/stable/modules/grid_search.html)

- Decide the range of hyperparameters you intend to explore
- Try using [`GridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) to perform brute force search
- Try using [`RandomizedSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV) for a random search
- Once you've chosen the best classifier and the best hyperparameter set, redo the learning curve.
Do you need more data or a better model?

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

In [ ]:
param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(2, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

clf = RandomForestClassifier(n_estimators=20)

model = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=40, n_jobs=-1, cv=3)
model.fit(X_train, y_train)

In [ ]:
model.best_score_

In [ ]:
model.score(X_test, y_test)

In [ ]:
best = model.best_estimator_

In [ ]:
best.fit(X_train, y_train)

In [ ]:
best.score(X_test, y_test)

In [ ]:
train_sizes, train_scores, test_scores = learning_curve(best, X, y, train_sizes=tsz, cv=3)

In [ ]:
fig = plt.figure()
plt.plot(train_sizes, train_scores.mean(axis=1), 'ro-', label="Train Scores")
plt.plot(train_sizes, test_scores.mean(axis=1), 'go-', label="Test Scores")
plt.title('Learning Curve: Logistic Regression')
plt.ylim((0.5, 1.0))
plt.legend()
plt.draw()
plt.show()